## Welcome to your notebook.


#### Run this cell to connect to your GIS and get started:

In [1]:
from arcgis.gis import GIS
gis = GIS("home")

/opt/conda/lib/python3.11/site-packages/arcgis/gis/__init__.py:731: UserWarning: You are logged on as ceco.ai with an administrator role, proceed with caution.
  warnings.warn(


In [2]:
import http.client
import pandas as pd
import json
from datetime import datetime, timedelta, timezone
from arcgis.features import GeoAccessor
from arcgis.geometry import SpatialReference
from arcgis.features import FeatureLayer

#### Now you are ready to start!

In [7]:
# Automatic air quality historical from Ambee
def fetch_ambee_data():
    # Create an HTTPS connection to the API server
    conn = http.client.HTTPSConnection("api.ambeedata.com")

    # Set the request headers including the API key and content type
    headers = {
        'x-api-key': "2eb0b3f4d3c3bd5fb7882e2aaca6145bd820cff96ccd771136c7874bc56812db",
        'Content-type': "application/json"
    }

    # Get the current UTC time and calculate the required dates
    end_date = datetime.now(timezone.utc)
    start_date = end_date - timedelta(days=2)

    # Format the dates to the required format for the API call
    formatted_start_date = start_date.strftime("%Y-%m-%d %H:%M:%S")
    formatted_end_date = end_date.strftime("%Y-%m-%d %H:%M:%S")

    # Encode the dates into URL encoding format, replacing spaces with %20 and colons with %3A
    encoded_start_date = formatted_start_date.replace(" ", "%20").replace(":", "%3A")
    encoded_end_date = formatted_end_date.replace(" ", "%20").replace(":", "%3A")

    # Construct the request URL with the encoded dates and coordinates
    request_path = f"/history/by-lat-lng?lat=57.7330&lng=-129.7677&from={encoded_start_date}&to={encoded_end_date}"

    # Send the GET request to the API
    conn.request("GET", request_path, headers=headers)

    # Get the response from the API
    response = conn.getresponse()
    data_historical_air_quality = response.read()
    json_data = json.loads(data_historical_air_quality.decode('utf-8'))

    # Get required data from json file and transfer to dataframe
    df = pd.DataFrame(json_data["data"])
    
    # tranfer DataFrame to Spatial DataFrame
    spatial_df = pd.DataFrame.spatial.from_xy(df, 'lng', 'lat',sr=SpatialReference(4326))
    feature_set = spatial_df.spatial.to_featureset()
    
    return feature_set

In [8]:
# Automatic weather historical from Ambee
def fetch_ambee_data1():
    # Create an HTTPS connection to the API server
    conn = http.client.HTTPSConnection("api.ambeedata.com")

    # Set the request headers including the API key and content type
    headers = {
        'x-api-key': "2eb0b3f4d3c3bd5fb7882e2aaca6145bd820cff96ccd771136c7874bc56812db",
        'Content-type': "application/json"
    }

    # Get the current UTC time and calculate the required dates
    end_date = datetime.now(timezone.utc)
    start_date = end_date - timedelta(days=2)

    # Format the dates to the required format for the API call
    formatted_start_date = start_date.strftime("%Y-%m-%d %H:%M:%S")
    formatted_end_date = end_date.strftime("%Y-%m-%d %H:%M:%S")

    # Encode the dates into URL encoding format, replacing spaces with %20 and colons with %3A
    encoded_start_date = formatted_start_date.replace(" ", "%20").replace(":", "%3A")
    encoded_end_date = formatted_end_date.replace(" ", "%20").replace(":", "%3A")

    # Construct the request URL with the encoded dates and coordinates
    request_path = f"/weather/history/by-lat-lng?lat=57.7330&lng=-129.7677&from={encoded_start_date}&to={encoded_end_date}"

    # Send the GET request to the API
    conn.request("GET", request_path, headers=headers)

    # Get the response from the API
    response = conn.getresponse()
    data_historical_air_quality = response.read()
    json_data = json.loads(data_historical_air_quality.decode('utf-8'))
    
    his = json_data['data']
    df = pd.DataFrame(his['history'])
    df = df.drop(columns=['summary','time','country_code','timezone'])
    df['lat'] = 57.733
    df['lng'] = -129.7677

    # tranfer DataFrame to Spatial DataFrame
    spatial_df = pd.DataFrame.spatial.from_xy(df, 'lng', 'lat',sr=SpatialReference(4326))
    feature_set = spatial_df.spatial.to_featureset()
    
    return feature_set

In [9]:
def upload_data_to_feature_layer(data, layer_url):
    #Check feature layer url
    layer = FeatureLayer(feature_layer_url)
    
    #upload
    layer.edit_features(adds=data)

In [10]:
#fetch air data
#fetch data
air_data = fetch_ambee_data()

#set url
feature_layer_url = "https://services6.arcgis.com/bKFxht3iaJpxZRCM/arcgis/rest/services/Group4_air/FeatureServer/0"

#upload data
upload_data_to_feature_layer(air_data, feature_layer_url)

In [11]:
#fetch weather data
#fetch data
weather_data = fetch_ambee_data1()
#set url
feature_layer_url = "https://services6.arcgis.com/bKFxht3iaJpxZRCM/arcgis/rest/services/Group4_weather/FeatureServer/0"

#upload data
upload_data_to_feature_layer(weather_data, feature_layer_url)

In [4]:

#from arcgis.gis import GIS
#from arcgis.features import FeatureLayer

#layer_url = "https://services6.arcgis.com/bKFxht3iaJpxZRCM/arcgis/rest/services/Group4_air/FeatureServer/0"
#feature_layer = FeatureLayer(layer_url, gis=gis)

#where_clause = "OBJECTID > 194"
#delete_result = feature_layer.delete_features(where=where_clause)
#print(delete_result)


{'deleteResults': [{'objectId': 195, 'uniqueId': 195, 'globalId': None, 'success': True}, {'objectId': 196, 'uniqueId': 196, 'globalId': None, 'success': True}, {'objectId': 197, 'uniqueId': 197, 'globalId': None, 'success': True}, {'objectId': 198, 'uniqueId': 198, 'globalId': None, 'success': True}, {'objectId': 199, 'uniqueId': 199, 'globalId': None, 'success': True}, {'objectId': 200, 'uniqueId': 200, 'globalId': None, 'success': True}, {'objectId': 201, 'uniqueId': 201, 'globalId': None, 'success': True}, {'objectId': 202, 'uniqueId': 202, 'globalId': None, 'success': True}, {'objectId': 203, 'uniqueId': 203, 'globalId': None, 'success': True}, {'objectId': 204, 'uniqueId': 204, 'globalId': None, 'success': True}, {'objectId': 205, 'uniqueId': 205, 'globalId': None, 'success': True}, {'objectId': 206, 'uniqueId': 206, 'globalId': None, 'success': True}, {'objectId': 207, 'uniqueId': 207, 'globalId': None, 'success': True}, {'objectId': 208, 'uniqueId': 208, 'globalId': None, 'succ